# 多头注意力机制 (Multi-Head Attention)

本 Notebook 讲解 Transformer 的核心组件：**多头注意力 (Multi-Head Attention)**。

---

## 1. 理论讲解

### 1.1 为什么需要多头？

单头注意力只能学习一种"关注模式"。但语言中的关系是多维度的：

| 关注维度 | 示例 |
|:--------|:-----|
| 语法结构 | 主语关注谓语 |
| 指代关系 | 代词关注其指代的名词 |
| 语义相似 | 同义词之间相互关注 |
| 位置关系 | 相邻词之间的关注 |

**类比**：就像用**多组不同的滤镜**观察同一张图像：
- 滤镜 1 捕捉边缘
- 滤镜 2 捕捉颜色
- 滤镜 3 捕捉纹理
- 最后将所有信息融合

多头注意力让模型能够**同时**从不同的表示子空间学习信息。

### 1.2 数学公式

**多头注意力**的定义：

$$\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, \dots, \text{head}_h)W^O$$

其中每个头的计算为：

$$\text{head}_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)$$

**参数维度**：
- $W_i^Q, W_i^K \in \mathbb{R}^{d_{model} \times d_k}$
- $W_i^V \in \mathbb{R}^{d_{model} \times d_v}$
- $W^O \in \mathbb{R}^{hd_v \times d_{model}}$
- 通常设置 $d_k = d_v = d_{model} / h$

### 1.3 维度变化流程

```
输入: [batch, seq, d_model]
         |
         v
    Linear(Q/K/V)
         |
         v
[batch, seq, d_model]
         |
         v
    view + transpose
         |
         v
[batch, n_heads, seq, d_k]   <-- 并行计算多头
         |
         v
  Scaled Dot-Product Attention
         |
         v
[batch, n_heads, seq, d_k]
         |
         v
    transpose + view
         |
         v
[batch, seq, d_model]   <-- 拼接所有头
         |
         v
    Linear(W_o)
         |
         v
输出: [batch, seq, d_model]
```

---

## 2. 代码实现

In [ ]:
import math
from typing import Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class MultiHeadAttention(nn.Module):
    """多头注意力机制。

    将输入投影到多个子空间，分别计算注意力后拼接输出。

    Attributes:
        d_model: 模型总维度。
        n_heads: 注意力头数。
        d_k: 每个头的维度 (d_model // n_heads)。
        W_q, W_k, W_v: Q/K/V 的线性投影层。
        W_o: 输出线性投影层。

    Example:
        >>> mha = MultiHeadAttention(d_model=512, n_heads=8)
        >>> x = torch.randn(2, 10, 512)
        >>> output, weights = mha(x, x, x)
        >>> assert output.shape == (2, 10, 512)
    """

    def __init__(self, d_model: int, n_heads: int, dropout: float = 0.0) -> None:
        """初始化多头注意力模块。

        Args:
            d_model: 模型总维度，必须能被 n_heads 整除。
            n_heads: 注意力头的数量。
            dropout: Dropout 概率，默认为 0.0。

        Raises:
            AssertionError: 当 d_model 不能被 n_heads 整除时。
        """
        super().__init__()
        assert d_model % n_heads == 0, f"d_model({d_model}) 必须能被 n_heads({n_heads}) 整除"

        self.d_model: int = d_model
        self.n_heads: int = n_heads
        self.d_k: int = d_model // n_heads

        # Q, K, V 线性投影
        self.W_q: nn.Linear = nn.Linear(d_model, d_model)
        self.W_k: nn.Linear = nn.Linear(d_model, d_model)
        self.W_v: nn.Linear = nn.Linear(d_model, d_model)

        # 输出投影
        self.W_o: nn.Linear = nn.Linear(d_model, d_model)

        self.dropout: nn.Dropout = nn.Dropout(p=dropout)
        self.scale: float = 1.0 / math.sqrt(self.d_k)

    def forward(
        self,
        query: torch.Tensor,
        key: torch.Tensor,
        value: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """前向传播。

        Args:
            query: Query 张量，形状 (batch, seq_q, d_model)。
            key: Key 张量，形状 (batch, seq_k, d_model)。
            value: Value 张量，形状 (batch, seq_k, d_model)。
            mask: 可选掩码，形状 (batch, 1, seq_q, seq_k) 或可广播形状。

        Returns:
            Tuple[torch.Tensor, torch.Tensor]:
                - output: 输出张量，形状 (batch, seq_q, d_model)。
                - attention_weights: 注意力权重，形状 (batch, n_heads, seq_q, seq_k)。
        """
        batch_size: int = query.size(0)

        # Step 1: 线性变换 -> (batch, seq, d_model)
        q: torch.Tensor = self.W_q(query)
        k: torch.Tensor = self.W_k(key)
        v: torch.Tensor = self.W_v(value)

        # Step 2: 重塑为多头 -> (batch, n_heads, seq, d_k)
        q = q.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        k = k.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)
        v = v.view(batch_size, -1, self.n_heads, self.d_k).transpose(1, 2)

        # Step 3: 计算注意力分数 -> (batch, n_heads, seq_q, seq_k)
        scores: torch.Tensor = torch.matmul(q, k.transpose(-2, -1)) * self.scale

        # Step 4: 应用掩码
        if mask is not None:
            scores = scores.masked_fill(mask == 1, float("-inf"))

        # Step 5: Softmax + Dropout
        attention_weights: torch.Tensor = F.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        # Step 6: 加权求和 -> (batch, n_heads, seq_q, d_k)
        context: torch.Tensor = torch.matmul(attention_weights, v)

        # Step 7: 拼接多头 -> (batch, seq_q, d_model)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Step 8: 输出投影
        output: torch.Tensor = self.W_o(context)

        return output, attention_weights

---

## 3. 验证与测试

In [ ]:
torch.manual_seed(42)

# 超参数
d_model: int = 512
n_heads: int = 8
batch_size: int = 2
seq_len: int = 10

# 初始化模型
mha = MultiHeadAttention(d_model=d_model, n_heads=n_heads)
print(f"模型参数量: {sum(p.numel() for p in mha.parameters()):,}")
print(f"每个头的维度 d_k: {mha.d_k}")

In [ ]:
# 创建输入
x: torch.Tensor = torch.randn(batch_size, seq_len, d_model)
print(f"输入形状: {x.shape}")

# 前向传播（自注意力：Q=K=V=x）
output, attention_weights = mha(x, x, x)

print(f"输出形状: {output.shape}")
print(f"注意力权重形状: {attention_weights.shape}")

In [ ]:
# 断言检查：输入输出形状一致
assert x.shape == output.shape, "输入输出形状不匹配！"
print("✓ 断言通过：input.shape == output.shape")

# 验证注意力权重归一化
weight_sum = attention_weights.sum(dim=-1)
assert torch.allclose(weight_sum, torch.ones_like(weight_sum), atol=1e-6)
print("✓ 断言通过：注意力权重每行之和为 1")

---

## 4. 可视化：不同头的注意力模式

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("8 个注意力头的关注模式", fontsize=14)

for i, ax in enumerate(axes.flat):
    sns.heatmap(
        attention_weights[0, i].detach().numpy(),
        ax=ax,
        cmap="viridis",
        cbar=False,
        square=True,
    )
    ax.set_title(f"Head {i+1}")
    ax.set_xlabel("Key")
    ax.set_ylabel("Query")

plt.tight_layout()
plt.show()

### 观察要点

- 不同的头学习到**不同的注意力模式**
- 有些头可能关注局部（对角线附近）
- 有些头可能关注全局（分布更均匀）
- 这种多样性让模型能捕获更丰富的语义关系

---

## 5. 总结

| 要点 | 说明 |
|:-----|:-----|
| **多头并行** | 将 d_model 拆分为 h 个子空间并行计算 |
| **参数效率** | 总参数量与单头相当 (4 × d_model²) |
| **表达能力** | 不同头捕获不同类型的依赖关系 |
| **维度不变** | 输入输出形状保持一致 |

**下一步**：在 `02-transformer-architecture/` 中，我们将学习如何将多头注意力组装成完整的 Encoder 和 Decoder。